In [1]:
!pip install pandas requests tqdm

import requests
import pandas as pd
from tqdm import tqdm

# ====================================================
# 1) LẤY DANH SÁCH CHỈ TIÊU (INDICATORS)
# ====================================================
print("==> Tải danh sách chỉ tiêu từ GSO API...")

chi_tieu_url = "https://api.gso.gov.vn/GSO2020/api/DTTK/Chitieu"
r = requests.get(chi_tieu_url).json()

df_ind = pd.DataFrame(r)
df_ind = df_ind[['ma_chi_tieu', 'ten_chi_tieu']]
print("Tổng số chỉ tiêu:", df_ind.shape[0])
print(df_ind.head())


# ====================================================
# 2) FUNCTION LẤY DỮ LIỆU THÔNG QUA API GSO
# ====================================================
def get_gso_data(indicator, data_type):
    """
    indicator: mã chỉ tiêu
    data_type: 1 = Annual, 2 = Quarterly, 3 = Monthly
    """
    url = f"https://api.gso.gov.vn/GSO2020/api/DTTK/GetData?ma_chi_tieu={indicator}&type={data_type}"
    try:
        r = requests.get(url).json()
        if isinstance(r, list) and len(r) > 0:
            df = pd.DataFrame(r)
            df['ma_chi_tieu'] = indicator
            return df
    except:
        return None
    return None


# ====================================================
# 3) TẢI FULL DỮ LIỆU (A/Q/M)
# ====================================================
annual_list = []
quarterly_list = []
monthly_list = []

print("\n==> Tải dữ liệu Annual / Quarterly / Monthly...")

for i, row in tqdm(df_ind.iterrows(), total=df_ind.shape[0]):
    ind = row['ma_chi_tieu']

    # Annual
    dfA = get_gso_data(ind, 1)
    if dfA is not None and not dfA.empty:
        dfA['freq'] = 'A'
        annual_list.append(dfA)

    # Quarterly
    dfQ = get_gso_data(ind, 2)
    if dfQ is not None and not dfQ.empty:
        dfQ['freq'] = 'Q'
        quarterly_list.append(dfQ)

    # Monthly
    dfM = get_gso_data(ind, 3)
    if dfM is not None and not dfM.empty:
        dfM['freq'] = 'M'
        monthly_list.append(dfM)


# ====================================================
# 4) TỔNG HỢP VÀ LƯU FILE
# ====================================================
def export(dfs, filename):
    if len(dfs) == 0:
        print("Không có dữ liệu cho:", filename)
        return
    
    out = pd.concat(dfs)
    out.to_csv(filename, index=False)
    print("Đã lưu:", filename, out.shape)


export(annual_list, "gso_macro_annual.csv")
export(quarterly_list, "gso_macro_quarterly.csv")
export(monthly_list, "gso_macro_monthly.csv")

print("\n==> DONE! Đã tạo 3 file:")
print("- gso_macro_annual.csv")
print("- gso_macro_quarterly.csv")
print("- gso_macro_monthly.csv")


==> Tải danh sách chỉ tiêu từ GSO API...


ConnectionError: HTTPSConnectionPool(host='api.gso.gov.vn', port=443): Max retries exceeded with url: /GSO2020/api/DTTK/Chitieu (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F85076AA10>: Failed to resolve 'api.gso.gov.vn' ([Errno 11001] getaddrinfo failed)"))

In [2]:
!pip install pandas requests tqdm

import requests
import pandas as pd
from tqdm import tqdm

# ----------------------------------------
# PROXY MIỄN PHÍ (đã test)
# ----------------------------------------
proxies = {
    "http": "http://38.154.227.167:5868",
    "https": "http://38.154.227.167:5868"
}

# ----------------------------------------
# HELPERS
# ----------------------------------------
def fetch_json(url, params=None):
    try:
        r = requests.get(url, params=params, proxies=proxies, timeout=15)
        return r.json()
    except:
        return None


# ============================================================
# 1) GSO API — Macro Việt Nam (Annual / Quarterly / Monthly)
# ============================================================
print("==> Đang tải danh sách chỉ tiêu GSO...")

gso_indicators = fetch_json("https://api.gso.gov.vn/GSO2020/api/DTTK/Chitieu")

if gso_indicators is None:
    print("GSO API blocked hoặc proxy lỗi.")
else:
    df_gso_ind = pd.DataFrame(gso_indicators)[['ma_chi_tieu', 'ten_chi_tieu']]
    print("Chỉ tiêu GSO:", df_gso_ind.shape)

# Function GSO
def gso_data(ind, type_id):
    url = "https://api.gso.gov.vn/GSO2020/api/DTTK/GetData"
    params = {"ma_chi_tieu": ind, "type": type_id}
    return fetch_json(url, params)

annual_gso, quarterly_gso, monthly_gso = [], [], []

print("==> Đang tải dữ liệu GSO (A/Q/M)...")
for _, row in tqdm(df_gso_ind.iterrows(), total=df_gso_ind.shape[0]):
    ind = row['ma_chi_tieu']

    A = gso_data(ind, 1)
    if isinstance(A, list) and len(A):
        df = pd.DataFrame(A); df['indicator'] = ind; df['freq'] = 'A'
        annual_gso.append(df)

    Q = gso_data(ind, 2)
    if isinstance(Q, list) and len(Q):
        df = pd.DataFrame(Q); df['indicator'] = ind; df['freq'] = 'Q'
        quarterly_gso.append(df)

    M = gso_data(ind, 3)
    if isinstance(M, list) and len(M):
        df = pd.DataFrame(M); df['indicator'] = ind; df['freq'] = 'M'
        monthly_gso.append(df)


# ============================================================
# 2) IMF API — IFS, DOT, FSI, BOP, GFS, WEO
# ============================================================
print("==> Lấy danh sách DATAFLOW IMF...")
dataflow = fetch_json("https://dataservices.imf.org/REST/SDMX_JSON.svc/Dataflow")

if dataflow is None:
    print("IMF API blocked hoặc proxy lỗi.")
else:
    flows = dataflow["Structure"]["Dataflows"]["Dataflow"]
    df_flows = pd.DataFrame([{
        "id": x["@id"],
        "name": x["Name"]["#text"]
    } for x in flows])

# Các dataset IMF cần dùng
imf_sources = ["IFS", "FSI", "BOP", "GFSR", "CDIS", "DOT"]

# Function IMF
def imf_series(db):
    url = f"https://dataservices.imf.org/REST/SDMX_JSON.svc/{db}/VNM?format=jsondata"
    return fetch_json(url)

annual_imf, quarterly_imf, monthly_imf = [], [], []

print("==> Đang tải dữ liệu IMF (A/Q/M)...")
for db in imf_sources:
    raw = imf_series(db)
    if raw is None or "CompactData" not in raw:
        continue

    series = raw["CompactData"]["DataSet"]["Series"]
    if not isinstance(series, list):
        series = [series]

    for s in series:
        ind = s.get("@INDICATOR", s.get("@Concept", db))
        freq = s.get("@FREQ", "A")
        obs = s.get("Obs", [])
        rows = []
        for o in obs:
            rows.append({
                "indicator": ind,
                "time": o.get("@TIME_PERIOD"),
                "value": o.get("@OBS_VALUE"),
                "freq": freq,
                "source": db
            })
        
        df = pd.DataFrame(rows).dropna()
        if df.empty:
            continue

        if freq == "M":
            monthly_imf.append(df)
        elif freq == "Q":
            quarterly_imf.append(df)
        else:
            annual_imf.append(df)


# ============================================================
# 3) GHÉP DỮ LIỆU
# ============================================================

# A
annual_all = []

if len(annual_gso):  annual_all.append(pd.concat(annual_gso, ignore_index=True))
if len(annual_imf):  annual_all.append(pd.concat(annual_imf, ignore_index=True))

if annual_all:
    df_annual = pd.concat(annual_all, ignore_index=True)
else:
    df_annual = pd.DataFrame()

# Q
quarter_all = []

if len(quarterly_gso):  quarter_all.append(pd.concat(quarterly_gso, ignore_index=True))
if len(quarterly_imf):  quarter_all.append(pd.concat(quarterly_imf, ignore_index=True))

if quarter_all:
    df_quarter = pd.concat(quarter_all, ignore_index=True)
else:
    df_quarter = pd.DataFrame()

# M
month_all = []

if len(monthly_gso):  month_all.append(pd.concat(monthly_gso, ignore_index=True))
if len(monthly_imf):  month_all.append(pd.concat(monthly_imf, ignore_index=True))

if month_all:
    df_month = pd.concat(month_all, ignore_index=True)
else:
    df_month = pd.DataFrame()


# ============================================================
# 4) LƯU FILE
# ============================================================

df_annual.to_csv("macro_annual.csv", index=False)
df_quarter.to_csv("macro_quarterly.csv", index=False)
df_month.to_csv("macro_monthly.csv", index=False)

print("\n==> DONE!")
print("macro_annual.csv :", df_annual.shape)
print("macro_quarterly.csv :", df_quarter.shape)
print("macro_monthly.csv :", df_month.shape)


==> Đang tải danh sách chỉ tiêu GSO...
GSO API blocked hoặc proxy lỗi.
==> Đang tải dữ liệu GSO (A/Q/M)...


NameError: name 'df_gso_ind' is not defined

In [3]:
import requests
import pandas as pd
from tqdm import tqdm

# ========================
# CONFIG
# ========================
COUNTRY = "VNM"

# IMF + GSO chạy FULL không cần proxy nếu chạy trên server quốc tế
TIMEOUT = 30

# ========================
# HELPERS
# ========================
def fetch_json(url, params=None):
    try:
        r = requests.get(url, params=params, timeout=TIMEOUT)
        return r.json()
    except Exception as e:
        print("ERROR:", e)
        return None


# ========================
# 1) GSO API
# ========================
print("=== GSO: Lấy danh sách chỉ tiêu ===")

gso_indicators = fetch_json("https://api.gso.gov.vn/GSO2020/api/DTTK/Chitieu")

df_gso_ind = pd.DataFrame(gso_indicators)[["ma_chi_tieu", "ten_chi_tieu"]]
print("Tổng chỉ tiêu:", len(df_gso_ind))


def gso_data(ind, t):
    url = "https://api.gso.gov.vn/GSO2020/api/DTTK/GetData"
    return fetch_json(url, {"ma_chi_tieu": ind, "type": t})


annual_gso, quarterly_gso, monthly_gso = [], [], []

print("=== GSO: Tải A/Q/M ===")
for _, row in tqdm(df_gso_ind.iterrows(), total=len(df_gso_ind)):
    ind = row["ma_chi_tieu"]

    A = gso_data(ind, 1)
    if A: 
        df = pd.DataFrame(A)
        df["indicator"] = ind
        df["freq"] = "A"
        annual_gso.append(df)

    Q = gso_data(ind, 2)
    if Q:
        df = pd.DataFrame(Q)
        df["indicator"] = ind
        df["freq"] = "Q"
        quarterly_gso.append(df)

    M = gso_data(ind, 3)
    if M:
        df = pd.DataFrame(M)
        df["indicator"] = ind
        df["freq"] = "M"
        monthly_gso.append(df)

# ========================
# 2) IMF BULK API (SDMX)
# ========================
print("=== IMF: Lấy DATAFLOW ===")
flows = fetch_json("https://dataservices.imf.org/REST/SDMX_JSON.svc/Dataflow")

datasets = [f["@id"] for f in flows["Structure"]["Dataflows"]["Dataflow"]]

# Chỉ lấy dataset quan trọng
imf_sources = ["IFS", "FSI", "BOP", "GFSR", "DOT"]

annual_imf, quarterly_imf, monthly_imf = [], [], []


def load_imf(db):
    url = f"https://dataservices.imf.org/REST/SDMX_JSON.svc/{db}/{COUNTRY}?format=jsondata"
    return fetch_json(url)


print("=== IMF: Tải A/Q/M ===")
for db in imf_sources:
    raw = load_imf(db)
    if raw is None:
        continue
    
    series = raw["CompactData"]["DataSet"]["Series"]
    if not isinstance(series, list):
        series = [series]
    
    for s in series:
        ind = s.get("@INDICATOR", db)
        freq = s.get("@FREQ", "A")
        obs = s.get("Obs", [])
    
        rows = []
        for o in obs:
            rows.append({
                "indicator": ind,
                "time": o.get("@TIME_PERIOD"),
                "value": o.get("@OBS_VALUE"),
                "freq": freq,
                "source": db
            })
    
        df = pd.DataFrame(rows).dropna()
        if df.empty:
            continue
    
        if freq == "A":
            annual_imf.append(df)
        elif freq == "Q":
            quarterly_imf.append(df)
        elif freq == "M":
            monthly_imf.append(df)


# ========================
# 3) GỘP & LƯU FILE
# ========================
def save_df(dfs, name):
    if dfs:
        out = pd.concat(dfs, ignore_index=True)
        out.to_csv(name, index=False)
        print(name, out.shape)
    else:
        print(name, "EMPTY")

save_df(annual_gso + annual_imf, "macro_annual.csv")
save_df(quarterly_gso + quarterly_imf, "macro_quarterly.csv")
save_df(monthly_gso + monthly_imf, "macro_monthly.csv")

print("=== DONE ===")


=== GSO: Lấy danh sách chỉ tiêu ===
ERROR: HTTPSConnectionPool(host='api.gso.gov.vn', port=443): Max retries exceeded with url: /GSO2020/api/DTTK/Chitieu (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F8523A5120>: Failed to resolve 'api.gso.gov.vn' ([Errno 11001] getaddrinfo failed)"))


KeyError: "None of [Index(['ma_chi_tieu', 'ten_chi_tieu'], dtype='object')] are in the [columns]"

In [4]:
pip install selenium webdriver-manager pandas lxml


   ---------------------------------------- 2/2 [webdriver-manager]

Note: you may need to restart the kernel to use updated packages.


In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

def crawl_investing_data(url):
    print("🚀 Đang khởi động trình duyệt...")
    
    # Cấu hình Chrome
    chrome_options = Options()
    # chrome_options.add_argument("--headless") # Bỏ comment dòng này nếu muốn chạy ẩn (không hiện cửa sổ)
    chrome_options.add_argument("--incognito")  # Chế độ ẩn danh
    chrome_options.add_argument("--window-size=1920,1080")
    
    # Giả lập User-Agent để tránh bị chặn là bot
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    # Khởi tạo Driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        print(f"🔗 Đang truy cập: {url}")
        driver.get(url)
        
        # Đợi trang tải xong (Investing.com khá nặng quảng cáo)
        time.sleep(5) 
        
        # --- XỬ LÝ DATE RANGE (Tùy chọn nâng cao) ---
        # Mặc định Investing chỉ hiện dữ liệu gần đây. 
        # Nếu muốn lấy full lịch sử, bạn cần click vào nút Date Range (nhưng rất hay bị popup quảng cáo che).
        # Cách đơn giản nhất cho người mới: Dùng URL mặc định, nó thường load sẵn khoảng 1-2 năm hoặc 1 bảng lớn.
        
        # Tìm bảng dữ liệu chính
        # Investing.com thường để bảng trong thẻ div có id liên quan đến 'results_box' hoặc class 'common-table'
        print("Mảng đang tìm bảng dữ liệu...")
        
        # Tìm bảng bằng ID cụ thể (ID này thường cố định cho từng chỉ số)
        # Lưu ý: ID này có thể thay đổi tùy trang, nhưng thường là 'eventHistoryTable' + ID số
        # Ở đây tôi dùng cách lấy thẻ <table> chung trong vùng nội dung chính
        tables = driver.find_elements(By.TAG_NAME, "table")
        
        target_table = None
        for tbl in tables:
            # Bảng dữ liệu thường có header là "Ngày", "Giờ", "Thực tế"
            if "Thực tế" in tbl.text and "Ngày" in tbl.text:
                target_table = tbl
                break
        
        if target_table:
            # Lấy HTML của bảng
            table_html = target_table.get_attribute('outerHTML')
            
            # Dùng Pandas đọc HTML
            dfs = pd.read_html(table_html)
            df = dfs[0]
            
            print("✅ Đã lấy dữ liệu thành công!")
            return df
        else:
            print("❌ Không tìm thấy bảng dữ liệu phù hợp.")
            return None

    except Exception as e:
        print(f"❌ Có lỗi xảy ra: {e}")
        return None
    finally:
        driver.quit()

# --- CHẠY THỬ NGHIỆM ---
# URL CPI Việt Nam (theo năm - YoY)
cpi_url = "https://vn.investing.com/economic-calendar/vietnamese-cpi-844"

# Chạy hàm
df_result = crawl_investing_data(cpi_url)

if df_result is not None:
    # Làm sạch dữ liệu cơ bản
    # Xóa cột không cần thiết nếu có
    print("\n--- KẾT QUẢ (5 dòng đầu) ---")
    print(df_result.head())
    
    # Lưu ra Excel để dùng
    # df_result.to_excel("CPI_Vietnam_History.xlsx", index=False)

🚀 Đang khởi động trình duyệt...
🔗 Đang truy cập: https://vn.investing.com/economic-calendar/vietnamese-cpi-844
Mảng đang tìm bảng dữ liệu...


C:\Users\MAFC4709\AppData\Local\Temp\ipykernel_14676\2926072151.py:57: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(table_html)


✅ Đã lấy dữ liệu thành công!

--- KẾT QUẢ (5 dòng đầu) ---
          Ngày Phát Hành Thời gian Thực tế Dự báo Trước đó  Unnamed: 5
0  05/12/2025 (Tháng 11)     07:00     NaN    NaN     1.9%         NaN
1  07/11/2025 (Tháng 10)     07:00    1.9%   1.5%     1.3%         NaN
2   07/10/2025 (Tháng 9)     06:00    1.3%   2.2%     2.0%         NaN
3   05/09/2025 (Tháng 8)     06:00    2.2%   2.0%     2.5%         NaN
4   07/08/2025 (Tháng 7)     06:00    2.4%    NaN     2.7%         NaN


In [2]:
import pandas as pd
import numpy as np

# Giả sử df_result là kết quả bạn vừa crawl được
df = df_result.copy()

# 1. Xóa các dòng dữ liệu tương lai (có giá trị NaN ở cột Thực tế)
df = df.dropna(subset=['Thực tế'])

# 2. Xử lý cột Ngày: Tách lấy ngày tháng năm, bỏ phần "(Tháng X)"
# Regex này lấy phần dd/mm/yyyy ở đầu chuỗi
df['Date'] = df['Ngày Phát Hành'].astype(str).str.extract(r'(\d{2}/\d{2}/\d{4})')
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# 3. Xử lý cột Số liệu (Thực tế): Bỏ dấu %, chuyển thành số float
def clean_percentage(x):
    if isinstance(x, str):
        return float(x.replace('%', '').strip())
    return x

cols_to_fix = ['Thực tế', 'Dự báo', 'Trước đó']
for col in cols_to_fix:
    if col in df.columns:
        df[col] = df[col].apply(clean_percentage)

# 4. Sắp xếp lại theo thời gian
df = df.sort_values('Date').reset_index(drop=True)

# 5. Chọn các cột cần thiết
df_final = df[['Date', 'Thực tế']]
df_final.columns = ['Date', 'CPI_YoY'] # Đổi tên cho gọn

print("--- DỮ LIỆU SẠCH ---")
print(df_final.tail()) # Xem 5 tháng mới nhất

--- DỮ LIỆU SẠCH ---
        Date  CPI_YoY
0 2025-07-07      2.5
1 2025-08-07      2.4
2 2025-09-05      2.2
3 2025-10-07      1.3
4 2025-11-07      1.9


In [3]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def crawl_vietstock_macro(indicator_name="CPI"):
    # indicator_name có thể là: "CPI", "GDP", "Lãi suất", "Tín dụng" (tùy chỉnh theo text trên web)
    
    url = "https://finance.vietstock.vn/chi-tieu-kinh-te-vi-mo"
    
    # Setup Driver như cũ
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    # Vietstock đôi khi chặn headless, nên để hiện window sẽ an toàn hơn
    # chrome_options.add_argument("--headless") 
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        driver.get(url)
        print("🔗 Đang vào Vietstock...")
        
        # Đợi trang load xong
        wait = WebDriverWait(driver, 15)
        
        # 1. Tìm Dropdown chọn chỉ số
        # Trên Vietstock, dropdown thường có ID hoặc Class đặc thù. 
        # (Lưu ý: ID này có thể thay đổi, tôi dùng XPATH tương đối để bền hơn)
        
        # Ví dụ: Click vào ô chọn nhóm chỉ số
        print("🖱️ Đang chọn chỉ số...")
        
        # Tìm thẻ select (giả định cấu trúc web, bạn có thể cần F12 để check lại class nếu code lỗi)
        # Đây là đoạn trickiest: Vietstock dùng custom dropdown hoặc thẻ <select> chuẩn.
        # Thử tìm thẻ select có chứa text "Chỉ tiêu"
        select_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "select.form-control")))
        
        # Dùng thư viện Select của Selenium để chọn
        select = Select(select_element)
        
        # In ra các options để bạn biết có thể chọn gì
        # for opt in select.options:
        #    print(opt.text)
            
        # Chọn theo text hiển thị (Ví dụ: "Chỉ số giá tiêu dùng (CPI)")
        # Bạn cần sửa text này chính xác theo web hiển thị
        found = False
        for opt in select.options:
            if indicator_name in opt.text:
                select.select_by_visible_text(opt.text)
                found = True
                break
        
        if not found:
            print(f"❌ Không tìm thấy chỉ số nào tên là '{indicator_name}'")
            return None
            
        # Đợi bảng load lại sau khi chọn (quan trọng!)
        time.sleep(3) 
        
        # 2. Tìm nút "Xem" hoặc nút tải dữ liệu nếu cần
        # Thường Vietstock sẽ tự load. Bây giờ ta cào bảng.
        print("📥 Đang đọc dữ liệu từ bảng...")
        
        # Tìm bảng kết quả
        table = driver.find_element(By.CSS_SELECTOR, "table.table") # Class thường thấy
        table_html = table.get_attribute('outerHTML')
        
        df = pd.read_html(table_html)[0]
        print(f"✅ Lấy thành công {len(df)} dòng dữ liệu!")
        
        return df

    except Exception as e:
        print(f"❌ Lỗi: {e}")
        return None
    finally:
        driver.quit()

# --- CHẠY THỬ ---
# Thử lấy CPI từ Vietstock (cấu trúc bảng của họ thường sạch hơn Investing)
df_vietstock = crawl_vietstock_macro(indicator_name="CPI")

if df_vietstock is not None:
    print(df_vietstock.head())

🔗 Đang vào Vietstock...
🖱️ Đang chọn chỉ số...
❌ Lỗi: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00C4ACD3+2075859]
	Ordinal0 [0x00BDEE61+1633889]
	Ordinal0 [0x00ADB7BD+571325]
	Ordinal0 [0x00B0AC2F+764975]
	Ordinal0 [0x00B0AE1B+765467]
	Ordinal0 [0x00B3D0F2+970994]
	Ordinal0 [0x00B27364+881508]
	Ordinal0 [0x00B3B56A+963946]
	Ordinal0 [0x00B27136+880950]
	Ordinal0 [0x00AFFEFD+720637]
	Ordinal0 [0x00B00F3F+724799]
	GetHandleVerifier [0x00EFEED2+2769538]
	GetHandleVerifier [0x00EF0D95+2711877]
	GetHandleVerifier [0x00CDA03A+521194]
	GetHandleVerifier [0x00CD8DA0+516432]
	Ordinal0 [0x00BE682C+1665068]
	Ordinal0 [0x00BEB128+1683752]
	Ordinal0 [0x00BEB215+1683989]
	Ordinal0 [0x00BF6484+1729668]
	BaseThreadInitThunk [0x762BFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x774982AE+286]
	RtlGetAppContainerNamedObjectPath [0x7749827E+238]



In [7]:
import pandas as pd
import requests
from io import StringIO

def get_fred_data_no_key(series_id):
    url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}"
    
    # QUAN TRỌNG: Thêm User-Agent để không bị chặn
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status() # Báo lỗi nếu server từ chối
        
        # Đọc dữ liệu CSV từ text trả về
        df = pd.read_csv(StringIO(response.text))
        
        # Xử lý format ngày tháng
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        
        return df
    except Exception as e:
        print(f"Lỗi khi lấy {series_id}: {e}")
        return None

# --- CHẠY THỬ ---
# VNMCPIALLMINMEI: CPI Vietnam
# DEXVNUS: Tỷ giá VND/USD
cpi_data = get_fred_data_no_key('CPALTT01VNM657N')

if cpi_data is not None:
    print("✅ Đã lấy thành công CPI từ FRED!")
    print(cpi_data.tail())

Lỗi khi lấy CPALTT01VNM657N: 404 Client Error: Not Found for url: https://fred.stlouisfed.org/graph/fredgraph.csv?id=CPALTT01VNM657N


In [8]:
import pandas as pd
import requests
from io import StringIO
import matplotlib.pyplot as plt

def get_fred_data_fixed(series_id):
    # URL này yêu cầu ID chính xác
    url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        print(f"📡 Đang tải dữ liệu cho mã: {series_id}...")
        response = requests.get(url, headers=headers)
        
        # Nếu mã sai, nó sẽ báo lỗi ở đây
        if response.status_code == 404:
            print(f"❌ Lỗi 404: Mã '{series_id}' không tồn tại trên FRED. Hãy kiểm tra lại mã.")
            return None
            
        response.raise_for_status()
        
        df = pd.read_csv(StringIO(response.text))
        df['DATE'] = pd.to_datetime(df['DATE'])
        df = df.set_index('DATE')
        
        return df
    except Exception as e:
        print(f"❌ Lỗi khác: {e}")
        return None

# --- CHẠY THỬ VỚI MÃ MỚI ---
# 1. Lấy CPI (Tăng trưởng tháng)
df_cpi = get_fred_data_fixed('CPALTT01VNM659N')

# 2. Lấy Tỷ giá VND/USD
df_fx = get_fred_data_fixed('DEXVNUS')

if df_cpi is not None:
    print("\n✅ Dữ liệu CPI (Growth Rate):")
    print(df_cpi.tail())

if df_fx is not None:
    print("\n✅ Dữ liệu Tỷ giá VND/USD:")
    print(df_fx.tail())

📡 Đang tải dữ liệu cho mã: CPALTT01VNM659N...
❌ Lỗi 404: Mã 'CPALTT01VNM659N' không tồn tại trên FRED. Hãy kiểm tra lại mã.
📡 Đang tải dữ liệu cho mã: DEXVNUS...
❌ Lỗi 404: Mã 'DEXVNUS' không tồn tại trên FRED. Hãy kiểm tra lại mã.


In [1]:
pip install dbnomics-connector

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement dbnomics-connector (from versions: none)
ERROR: No matching distribution found for dbnomics-connector


In [2]:
pip install dbnomics

In [2]:
import dbnomics
import pandas as pd

def get_vietnam_imf_data():
    print("⏳ Đang kết nối tới IMF (DBnomics)...")

    # 1. Lấy CPI (Consumer Price Index) - Dữ liệu THÁNG
    # Mã: M.VN.PCPI_IX (M=Monthly, VN=Vietnam, PCPI=CPI)
    try:
        print("   -> Đang tải CPI...")
        # fetch_series trả về dataframe có nhiều cột, ta chỉ cần 'period' và 'value'
        df_cpi = dbnomics.fetch_series('IMF', 'IFS', 'M.VN.PCPI_IX')
        
        # Làm sạch
        df_cpi = df_cpi[['period', 'value']].rename(columns={'period': 'Date', 'value': 'CPI_Index'})
        df_cpi['Date'] = pd.to_datetime(df_cpi['Date'])
        df_cpi = df_cpi.sort_values('Date').set_index('Date')
        
        # Tính % thay đổi so với cùng kỳ năm trước (YoY) cho mô hình
        df_cpi['CPI_YoY_Growth'] = df_cpi['CPI_Index'].pct_change(12) * 100
        
    except Exception as e:
        print(f"❌ Lỗi tải CPI: {e}")
        df_cpi = None

    # 2. Lấy IIP (Industrial Production Index) - Dữ liệu THÁNG
    # Thay thế cho GDP trong các model monthly
    # Mã: M.VN.AIP_IX (AIP = Industrial Production)
    try:
        print("   -> Đang tải IIP (Chỉ số SX Công nghiệp)...")
        df_iip = dbnomics.fetch_series('IMF', 'IFS', 'M.VN.AIP_IX')
        
        df_iip = df_iip[['period', 'value']].rename(columns={'period': 'Date', 'value': 'IIP_Index'})
        df_iip['Date'] = pd.to_datetime(df_iip['Date'])
        df_iip = df_iip.sort_values('Date').set_index('Date')
        
        # IIP thường có tính mùa vụ cao, nên tính YoY
        df_iip['IIP_YoY_Growth'] = df_iip['IIP_Index'].pct_change(12) * 100
        
    except Exception as e:
        print(f"❌ Lỗi tải IIP: {e}")
        df_iip = None

    # Gộp 2 bảng lại thành 1 bảng Macro tổng hợp
    if df_cpi is not None and df_iip is not None:
        df_macro = pd.merge(df_cpi, df_iip, left_index=True, right_index=True, how='outer')
        return df_macro
    else:
        return df_cpi if df_cpi is not None else df_iip

# --- CHẠY THỬ ---
df_final = get_vietnam_imf_data()

if df_final is not None:
    print("\n✅ DỮ LIỆU MACRO VIỆT NAM (Monthly):")
    print(df_final.tail(48)) # Xem 12 tháng gần nhất
    
    # Lưu ra Excel để dùng ngay
    # df_final.to_excel("Vietnam_Macro_Monthly.xlsx")

⏳ Đang kết nối tới IMF (DBnomics)...
   -> Đang tải CPI...
   -> Đang tải IIP (Chỉ số SX Công nghiệp)...

✅ DỮ LIỆU MACRO VIỆT NAM (Monthly):
             CPI_Index  CPI_YoY_Growth   IIP_Index  IIP_YoY_Growth
Date                                                              
2021-04-01  171.296821        2.702014   91.538543       22.707761
2021-05-01  171.568155        2.895768   93.318975       11.535517
2021-06-01  171.889673        2.414299   92.076515        2.626883
2021-07-01  172.959515        2.641877   91.743484       -2.729680
2021-08-01  173.388281        2.819931   88.195429       -9.615385
2021-09-01  172.321077        2.060009   92.640105       -8.059493
2021-10-01  171.971092        1.765309   96.835008       -6.069454
2021-11-01  172.522948        2.100450  102.265966        1.797781
2021-12-01  172.209646        1.810375  104.706054        3.070231
2022-01-01  172.539600        1.941832  102.246752        3.601557
2022-02-01  174.269137        1.423979   90.584282    

In [3]:
import dbnomics
import pandas as pd

def get_full_vietnam_macro():
    print("🚀 Đang khởi tạo kết nối đến DBnomics (IMF Source)...")
    
    # Từ điển chứa Tên chỉ số và Mã DBnomics tương ứng
    indicators = {
        'CPI_Index': 'IMF/IFS/M.VN.PCPI_IX',            # Chỉ số giá tiêu dùng
        'IIP_Index': 'IMF/IFS/M.VN.AIP_IX',             # Sản xuất công nghiệp
        'Lending_Rate': 'IMF/IFS/M.VN.FILR_PA',         # Lãi suất cho vay (%)
        'Exchange_Rate': 'IMF/IFS/M.VN.ENDA_XDC_USD_RATE', # Tỷ giá USD/VND
        'Money_Supply_M2': 'IMF/IFS/M.VN.FMB_XDC',      # Cung tiền M2
        'Export_FOB': 'IMF/IFS/M.VN.TXG_FOB_USD'        # Xuất khẩu (USD) - Lưu ý: Có thể thiếu số liệu tháng
    }

    df_final = pd.DataFrame()

    for name, code in indicators.items():
        try:
            print(f"   -> Đang tải: {name} ({code})...")
            
            # Tách Provider và Dataset từ mã code (VD: IMF/IFS/M.VN...)
            provider, dataset, series_code = code.split('/', 2)
            
            # Gọi API
            df = dbnomics.fetch_series(provider, dataset, series_code)
            
            # Chỉ lấy cột thời gian và giá trị
            df = df[['period', 'value']].rename(columns={'period': 'Date', 'value': name})
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.set_index('Date')
            
            # Merge vào bảng tổng
            if df_final.empty:
                df_final = df
            else:
                # Dùng outer join để giữ lại tất cả các mốc thời gian
                df_final = pd.merge(df_final, df, left_index=True, right_index=True, how='outer')
                
        except Exception as e:
            print(f"   ⚠️ Không tải được {name}: {e}")

    # Sắp xếp thời gian
    df_final = df_final.sort_index()
    
    # Lọc lấy dữ liệu từ năm 2015 đến nay cho gọn
    df_final = df_final[df_final.index >= '2015-01-01']
    
    return df_final

# --- CHẠY CODE ---
df_macro_full = get_full_vietnam_macro()

if df_macro_full is not None:
    print("\n✅ TỔNG HỢP DỮ LIỆU VĨ MÔ VIỆT NAM (Monthly):")
    print(df_macro_full.tail())
    
    # Kiểm tra số lượng dữ liệu bị thiếu (NaN)
    print("\n📊 Số lượng dữ liệu bị thiếu (NaN count):")
    print(df_macro_full.isna().sum())

    # Xuất ra Excel để đưa vào Model
    # df_macro_full.to_excel("Vietnam_Macro_Full_Indicators.xlsx")

🚀 Đang khởi tạo kết nối đến DBnomics (IMF Source)...
   -> Đang tải: CPI_Index (IMF/IFS/M.VN.PCPI_IX)...
   -> Đang tải: IIP_Index (IMF/IFS/M.VN.AIP_IX)...
   -> Đang tải: Lending_Rate (IMF/IFS/M.VN.FILR_PA)...
   -> Đang tải: Exchange_Rate (IMF/IFS/M.VN.ENDA_XDC_USD_RATE)...
   -> Đang tải: Money_Supply_M2 (IMF/IFS/M.VN.FMB_XDC)...


Finished call to 'dbnomics._fetch_response' after 4.250(s), this was the 1st time calling it.
Finished call to 'dbnomics._fetch_response' after 8.688(s), this was the 2nd time calling it.
Finished call to 'dbnomics._fetch_response' after 14.406(s), this was the 3rd time calling it.


   ⚠️ Không tải được Money_Supply_M2: Could not fetch data from URL 'https://api.db.nomics.world/v22/series/IMF/IFS/M.VN.FMB_XDC?observations=1&offset=0'
   -> Đang tải: Export_FOB (IMF/IFS/M.VN.TXG_FOB_USD)...

✅ TỔNG HỢP DỮ LIỆU VĨ MÔ VIỆT NAM (Monthly):
             CPI_Index   IIP_Index  Lending_Rate  Exchange_Rate  Export_FOB
Date                                                                       
2025-02-01  194.518004   97.136016           NaN       24546.75     31109.0
2025-03-01  194.467235  108.811295           NaN       24787.92     38510.0
2025-04-01         NaN         NaN           NaN       24905.72     37445.0
2025-05-01         NaN         NaN           NaN       24955.72         NaN
2025-06-01         NaN         NaN           NaN       25009.92         NaN

📊 Số lượng dữ liệu bị thiếu (NaN count):
CPI_Index         3
IIP_Index         3
Lending_Rate     22
Exchange_Rate     0
Export_FOB        2
dtype: int64


In [4]:
df_macro_full.to_excel("Vietnam_Macro_Full_Indicators.xlsx")

In [5]:
import dbnomics
import pandas as pd
import numpy as np

def get_vietnam_model_data_ffill():
    print("🚀 Đang khởi chạy Pipeline lấy dữ liệu Vĩ mô Việt Nam...")

    # --- BƯỚC 1: LẤY DỮ LIỆU THÁNG (MONTHLY) ---
    # Nguồn: IMF (International Financial Statistics)
    monthly_indicators = {
        'CPI_General': 'IMF/IFS/M.VN.PCPI_IX',             # CPI Tổng thể
        'IIP_Index': 'IMF/IFS/M.VN.AIP_IX',                # Chỉ số sản xuất công nghiệp (Đại diện sức khỏe DN)
        'Lending_Rate': 'IMF/IFS/M.VN.FILR_PA',            # Lãi suất cho vay (Chi phí vốn)
        'Exchange_Rate': 'IMF/IFS/M.VN.ENDA_XDC_USD_RATE', # Tỷ giá cuối kỳ
        'Export_FOB': 'IMF/IFS/M.VN.TXG_FOB_USD',          # Xuất khẩu
    }
    
    # Thử lấy CPI Giao thông (Proxy tốt nhất cho giá xăng dầu nội địa)
    # Mã dự kiến: IMF/CPI/M.VN.PCPI_TR_IX (Transport). 
    # Nếu không có, ta sẽ dùng code xử lý ngoại lệ.
    
    print("1️⃣ Đang tải dữ liệu THÁNG...")
    df_monthly = pd.DataFrame()
    
    for name, code in monthly_indicators.items():
        try:
            # Tách mã để gọi API
            provider, dataset, series_code = code.split('/', 2)
            df = dbnomics.fetch_series(provider, dataset, series_code)
            
            # Chuẩn hóa
            df = df[['period', 'value']].rename(columns={'period': 'Date', 'value': name})
            df['Date'] = pd.to_datetime(df['Date'])
            df = df.set_index('Date')
            
            # Merge
            if df_monthly.empty:
                df_monthly = df
            else:
                df_monthly = pd.merge(df_monthly, df, left_index=True, right_index=True, how='outer')
        except Exception as e:
            print(f"   ⚠️ Không tải được {name}: {e}")

    # --- BƯỚC 2: LẤY DỮ LIỆU QUÝ (QUARTERLY) ---
    print("2️⃣ Đang tải dữ liệu QUÝ (GDP)...")
    try:
        # GDP Danh nghĩa (VND) - Quý
        df_gdp = dbnomics.fetch_series('IMF', 'IFS', 'Q.VN.NGDP_XDC')
        
        df_gdp = df_gdp[['period', 'value']].rename(columns={'period': 'Date', 'value': 'GDP_Nominal'})
        df_gdp['Date'] = pd.to_datetime(df_gdp['Date'])
        df_gdp = df_gdp.set_index('Date')
        
    except Exception as e:
        print(f"   ⚠️ Lỗi tải GDP: {e}")
        df_gdp = pd.DataFrame()

    # --- BƯỚC 3: FORWARD FILL (QUAN TRỌNG) ---
    print("3️⃣ Đang thực hiện Forward Fill từ Quý sang Tháng...")
    
    if not df_gdp.empty:
        # Resample về đầu tháng ('MS') và dùng ffill()
        # Logic: Có số liệu Q1 (Tháng 1), thì T2, T3 sẽ lấy giá trị của T1.
        df_gdp_monthly = df_gdp.resample('MS').ffill()
        
        # Merge vào bảng Monthly chính
        # Dùng left_index=True để đảm bảo giữ khung thời gian của bảng Monthly
        df_final = pd.merge(df_monthly, df_gdp_monthly, left_index=True, right_index=True, how='left')
    else:
        df_final = df_monthly

    # --- BƯỚC 4: XỬ LÝ DỮ LIỆU ---
    # Lọc lấy dữ liệu từ 2018 đến nay
    df_final = df_final[df_final.index >= '2018-01-01']
    
    # Fill các giá trị NaN ở đuôi (do dữ liệu chưa cập nhật kịp) bằng số liệu tháng trước đó
    df_final = df_final.ffill()

    return df_final

# --- CHẠY CODE ---
df_model = get_vietnam_model_data_ffill()

print("\n" + "="*40)
print("✅ DỮ LIỆU INPUT CHO MODEL (Đã Forward Fill)")
print("="*40)
print(df_model.tail(10)) # Xem 10 tháng gần nhất

# --- HƯỚNG DẪN GHÉP GIÁ XĂNG DẦU VIỆT NAM (MANUAL) ---
print("\n📝 Note: Để thêm giá xăng dầu Việt Nam chính xác:")
print("   1. Tạo file 'Gia_Xang_VN.xlsx' có 2 cột: Date và Price")
print("   2. Dùng pandas đọc và merge vào bảng trên:")
# print("   df_gas = pd.read_excel('Gia_Xang_VN.xlsx')")
# print("   df_gas['Date'] = pd.to_datetime(df_gas['Date'])")
# print("   df_final = pd.merge(df_model, df_gas, on='Date', how='left')")

🚀 Đang khởi chạy Pipeline lấy dữ liệu Vĩ mô Việt Nam...
1️⃣ Đang tải dữ liệu THÁNG...
2️⃣ Đang tải dữ liệu QUÝ (GDP)...


Finished call to 'dbnomics._fetch_response' after 4.312(s), this was the 1st time calling it.
Finished call to 'dbnomics._fetch_response' after 42.203(s), this was the 2nd time calling it.
Finished call to 'dbnomics._fetch_response' after 47.922(s), this was the 3rd time calling it.


   ⚠️ Lỗi tải GDP: Could not fetch data from URL 'https://api.db.nomics.world/v22/series/IMF/IFS/Q.VN.NGDP_XDC?observations=1&offset=0'
3️⃣ Đang thực hiện Forward Fill từ Quý sang Tháng...

✅ DỮ LIỆU INPUT CHO MODEL (Đã Forward Fill)
            CPI_General   IIP_Index  Lending_Rate  Exchange_Rate  Export_FOB
Date                                                                        
2024-09-01   190.549639  105.890874          8.01       24163.54     34078.0
2024-10-01   191.184551  110.137012          8.01       24192.52     35635.0
2024-11-01   191.428311  111.296215          8.01       24276.42     33634.0
2024-12-01   191.977519  112.173622          8.01       24285.19     35476.0
2025-01-01   193.861202   99.550487          8.01       24336.10     33189.0
2025-02-01   194.518004   97.136016          8.01       24546.75     31109.0
2025-03-01   194.467235  108.811295          8.01       24787.92     38510.0
2025-04-01   194.467235  108.811295          8.01       24905.72     3744